In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import os
print(os.getcwd())


In [ ]:
# look at the first ten thousand bytes to guess the character encoding
with open("PakistanSuicideAttacks.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result)

In [8]:
# read in our dat
suicide_attacks = pd.read_csv("PakistanSuicideAttacks.csv", 
                              encoding='Windows-1252')

In [ ]:
suicide_attacks.shape

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities



I can see some problems due to inconsistent data entry: 'Lahore' and 'Lahore ', for example, or 'Lakki Marwat' and 'Lakki marwat' and 'Tank', 'Tank '. 

In [11]:
# convert to lower case
suicide_attacks['City'] = suicide_attacks['City'].str.lower()
# remove trailing white spaces
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

It seems there are still some inconsistencies: for instance, "d. i khan" and "d.i khan" should likely be standardized to the same format. (I double-checked, and "D.G Khan" is a different city, so it shouldn’t be combined with "D.I Khan.")

To streamline this process, I’ll use the fuzzywuzzy package to identify similar strings. Although this dataset is small enough to correct by hand, manually fixing errors doesn’t scale well. (Would you want to correct thousands of errors by hand? Automating tasks early on is usually wise—and it’s satisfying!)

Fuzzy Matching: This is the process of finding strings that closely resemble a target string. Generally, the fewer changes needed to transform one string into another, the "closer" they are. For example, "apple" and "snapple" differ by just two characters, while "in" and "on" differ by only one character. While fuzzy matching isn’t always perfect, it can significantly reduce manual work.

fuzzywuzzy provides a similarity ratio for two strings. The closer this ratio is to 100, the smaller the edit distance. Here, we’ll retrieve the ten city names with the highest similarity to "d.i khan."

In [ ]:
# get the top 10 closest matches to "d.i khan"
matches = fuzzywuzzy.process.extract("d.i khan", cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches



We notice that two entries for cities are almost identical to "D.I Khan": "d. i khan" and "d.i khan." Additionally, there's "D.G Khan," a separate city, with a similarity ratio of 88. To avoid mistakenly changing "D.G Khan" to "D.I Khan," we'll update only those rows in our City column that have a similarity ratio greater than 90, replacing them with "D.I Khan."

I'll create a function for this. Writing a reusable function is beneficial if you expect to perform a task multiple times—it saves time, reduces repetitive code, and minimizes the risk of errors.

In [14]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [ ]:
# use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="d.i khan")

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities



In [ ]:
# get the top 10 closest matches to "d.i khan"
matches = fuzzywuzzy.process.extract("kurram agency", cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

In [ ]:
# use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="kurram agency")

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities
